# Introduction

Some time ago I was trying to find an interesting data science project to tackle, and I remembered reading about the possibility of collecting and analyzing tweets posted on the social networking site Twitter, though I had no idea how to go about doing this at the time. The task seemed interesting but a little daunting, as I had never used Twitter before, and I thought the ability to mine tweets would require writing some very complex code that was potentially way over my head. However, I recently decided to read up a little more on the subject to see if I could pull it off, and this post details what I’ve been able to find so far. I hope it’s interesting or of some use to others!

The process of collecting tweets is carried out through the use of Twitters Application Programming Interface, referred to as an API. It’s important to note that API’s are not unique to Twitter; other sites have them as well. For example, YouTube, Google, and Reddit all have their own API’s, which allows one to pull useful information from these sites regarding, in the case of YouTube, user video subscriptions, the set of videos a channel has chosen to feature, etc. As we’ll see, in the case of Twitter, this information includes tweet text as well as geographic location of the user, number of Twitter followers, etc. The cool thing about Twitter’s API is that you can stream tweets about a particular topic in real time. Below, I’ll detail how to set it up an account to connect Twitter’s API, and show the code I use to mine tweets using Python.

# Getting the API Keys

Before start collecting tweets, we must first complete a few preliminary steps in order to use the Twitter API. The first step in the process is to get the necessary API “keys” which allow us to connect to the API. API keys are simply long strings of letter-number combinations that act sort of like a set of passwords. We can get them by doing the following:

* Create a Twitter account, and choose a username and password. Under profile setting, go to “mobile” and enter in your phone number; this is necessary to proceed to the next step. You will be sent an activation text, and you can enter it in the prompt on your computer screen.

* Go to http://apps.twitter.com and click “create a new app”. You will be prompted to enter a name for your “application” and a description of it. This is not really important for what we want to do, so I simply called mine “CoolNewApp”, and placed “App for streaming tweets” in the description line. In the “website” and “callback URL” prompts, just enter a valid domain name. I chose something simple like http://www.test.com. Once this is done, agree to the developer agreement at the bottom of the screen.

* When the above steps are done correctly, you’ll be taken to a page showing your application details. Click on the “Keys and Access Tokens” tab. Click “Create my access token” at the bottom of the page. Copy your Consumer Key (API Key), Consumer Secret (API Secret), Access Token, and Access Token Secret. These are your passwords to connect to the API.

In my case, the keys look something like this:

# The Tweepy Library

The section above details how to gain access to the Twitter API. Now that Twitter has allowed us to mine tweets, we have to figure out how to use Python to actually do it. Luckily for us, Python has a nice library called “tweepy” which allows us to do just that. To install tweepy, simply run

In [ ]:
pip install tweepy

from the terminal command line.

# Connecting to the API using Tweepy

Okay, now that we have tweepy, all we need is a little bit of code to start collecting tweets in real time. In my searching, I was able to find three very useful resources to help us do this:

* [Adil Moujahid’s Blog](http://adilmoujahid.com/posts/2014/07/twitter-analytics/)
* [Marco Bonzanini’s Blog](https://degravek.github.io/project_api)
* [Sentdex’s YouTube Channel](https://www.youtube.com/watch?v=pUUxmvvl2FE)

The code below was put together using a mixture of tutorials from these three resources. I’ll go through the different sections of code step by step and try to explain in turn what each does.

In [ ]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

access_token = "766363654925193216-fsh7W0LrsL7GzNO5mgSLWHl8S91NaPj"
access_secret = "39SEIEFB2b43A8UfyUDwTn3I0pdAm3wxASjuwu6YMUA7x"
consumer_key = "hBHpC1CHL16etnzQeoa9v8oA9"
consumer_secret = "plSpmxf0Ipk6hMEBX50M7DGdbi93x3lvJX5bV41ATJpZgiu0pO"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            tweet = data.split(',"text":"')[1].split('","source')[0]
            print tweet

            with open('ufc.json', 'a') as f:
                f.write(data)
                return True
        except:
            pass
 
    def on_error(self, status):
        print(status)
        return True
 
twitterStream = Stream(auth, MyListener())
twitterStream.filter(track=['#UFC', 'UFC', 'Ultimate Fighting'])

The beginning of the script is where we import tweepy and a few of its methods called Stream, StreamListener, and OAuthHandler. OAuthHandler will allow us to access the API using our authentication keys we set up in the Getting the API Keys section above. StreamListener is responsible for collecting tweets in real time, and Stream has a method called “filter” which allows us to filter worldwide tweets by certain keywords that we can set. Below this is where we finally set the Access Token, Access Key, Consumer Token, and Consumer Key values that we created earlier. We then define a new class called MyListener which contains two calls - one to on_data, and one to on_error. If possible, on_data takes a tweet, prints the text, and appends it to a file called ufc.json (the JSON extension stands for JavaScript Object Notation). If on_data cannot do this, on_error prints the error and continues listening for new tweets. Lastly, we set the filter keywords. In our case, these keywords are ‘#UFC’, ‘UFC’, and ‘Ultimate Fighting’.

After running the above code, one will notice that each tweet actually contains a huge amount of information, and that all of it is saved to the JSON output file. The figure below shows the anatomy of a single tweet as an easy-to-read graphic, borrowed from Wei Xu’s API tutorial found [here](http://socialmedia-class.org/twittertutorial.html).

![tweet](https://github.com/degravek/notebooks/blob/master/images/tweet.png?raw=true)

Each tweet contains information regarding user name, ID number, language spoken, a time stamp, text contents, location, the number of users an individual is following, among many other things. All of this data tends to accumulate very rapidly, especially if we filter tweets by a very popular tag. If we’re only interested in collecting a few bits of information from each tweet (for example, tweet text and user location), we can chop the contents down quite a bit before saving to our output file. In the next section I show how this can be done using an alternative implementation of the Python code above.

# An Alternative Implementation

Below is a different implementation of the MyListener class which allows one to chop tweets down by keyword before saving.

In [ ]:
class MyListener(StreamListener):

    def on_data(self, data):
        try:
            tweet    = data.split(',"text":"')[1].split('","source')[0]
            language = data.split(',"lang":"')[1].split('","contributors_enabled')[0]
            timezone = data.split(',"time_zone":"')[1].split('","geo_enabled')[0]
            print tweet

            saveThis = language + '::' + timezone + '::' + tweet
            saveFile = open('ufc.csv','a')
            saveFile.write(saveThis)
            saveFile.write('\n')
            saveFile.close()
            return True
        except:
            pass

    def on_error(self, status):
        print(status)

twitterStream = Stream(auth, MyListener())
twitterStream.filter(track=['#UFC', 'UFC', 'Ultimate Fighting'])

In this case, we use the split command to isolate the tweet text, language, and time zone of each user. This is shown in the first few lines of code. For example, the line

In [ ]:
timezone = data.split(',"time_zone":"')[1].split('","geo_enabled')[0]

says “look through the full tweet and give me all contents to the right of substring ‘,”time_zone”:”’, and everything to the left of substring ‘”,”geo_enabled’. After isolating tweet text, language, and time zone, we can combine the three into one line, each separated by a double colon ‘::’ using

In [ ]:
saveThis = language + '::' + timezone + '::' + tweet

These are then saved to a .csv (comma separated values) file, which can be read in later for analysis.

Concluding Remarks
In this post I’ve covered in detail how to connect to Twitter’s application programming interface using Python’s tweepy library. In the next post, I’ll do some tweet collecting myself, and explore what kinds of information we can gather from Twitter users all over the world. Thanks for following along!

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>